In [1]:
import os 
import math
import random
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from numpy import count_nonzero

dir_ = '../data/'
group_dir_ = '../data/groups/high'

In [2]:
file_name = 'normalized_to_rating_filter_track_5_user_100.csv'
svd = pd.read_pickle(os.path.join(dir_, 'prediction_svd_top_N_' + file_name[:-3] + 'pkl'))
test = pd.read_pickle(os.path.join(dir_, 'test_' + file_name[:-3] + 'pkl'))
test.sort_values(by=['uid','tid'])

file_name = 'normalized_popularity_filter_track_5_user_100.csv'
pop_count = pd.read_pickle(os.path.join(dir_, 'prediction_popularity_count_top_N_' + file_name[:-3] + 'pkl'))
pop_rating = pd.read_pickle(os.path.join(dir_, 'prediction_popularity_rating_top_N_' + file_name[:-3] + 'pkl'))

num_user = len(svd['uid'].unique())
num_user

220

In [3]:
groups_n =[]
group_sizes = ['2', '3', '4', '5']
for i in range(len(group_sizes)):
    groups = np.loadtxt(os.path.join(group_dir_, 'group_' + group_sizes[i] + '.csv'), delimiter=',')
    groups_n.append(groups)

In [4]:
for i in groups_n:
    print(len(i[0]))

2
3
4
5


In [5]:
svd['count'] = 1
svd

,uid,tid,rating,count
0,0.0,0.0,2.493650,1
1,0.0,6.0,2.333071,1
2,0.0,8.0,2.713340,1
3,0.0,9.0,1.949588,1
4,0.0,12.0,2.477737,1
...,...,...,...,...
11741994,219.0,54964.0,1.816855,1
11741995,219.0,54965.0,2.315346,1
11741996,219.0,54966.0,1.954432,1
11741997,219.0,54967.0,2.071366,1


In [6]:
pop_count = pop_count.sort_values(by=['count'],  ascending=False)
pop_count

,tid,count,rating
166,166,1882,333.50
6310,6338,1589,92.25
17316,17377,1555,43.50
6254,6282,1396,17.50
13055,13104,1340,33.50
...,...,...,...
25819,25941,1,0.25
43440,43743,1,0.25
54130,54607,1,0.25
27486,27620,1,0.25


In [7]:
# Linear
# size = int(len(pop_rating) // 5)
# pop_normalized = pd.DataFrame()
# for i in range(5):
#     if i == 0:
#         pop = pop_count[:size].copy()
#     elif i == 4:
#         pop = pop_count[size*4:].copy()
#     else:
#         pop = pop_count[size*i:size*(i+1)].copy()
#     pop['rating'] = (5 - i)
#     pop_normalized = pop_normalized.append(pop)
# pop_normalized = pop_normalized[['tid', 'rating']]

In [8]:
# min_max
max_rating = pop_count.iloc[0]['count']
pop_normalized = pop_count.copy()
pop_normalized = pop_normalized[['tid', 'count']]
pop_normalized['rating'] = pop_normalized['count']
pop_normalized['rating'] /= max_rating
pop_normalized['rating'] *= 5
pop_normalized

,tid,count,rating
166,166,1882,5.000000
6310,6338,1589,4.221573
17316,17377,1555,4.131243
6254,6282,1396,3.708820
13055,13104,1340,3.560043
...,...,...,...
25819,25941,1,0.002657
43440,43743,1,0.002657
54130,54607,1,0.002657
27486,27620,1,0.002657


In [77]:
lambda_ = 0.8

In [78]:
top_n_size = 100
top_n_lists_g_hybrid = []
top_n_lists_g_pop = []
top_n_lists_g_svd = []

for groups in groups_n:

    group_size = len(groups[0])
    top_n_lists_hybrid = []
    top_n_lists_pop = []
    top_n_lists_svd = []

    for group in groups:

        predictions = []
        rating_table = pd.DataFrame() 
        for member in group:
            prediction = svd[svd['uid'] == member].copy()
            if len(rating_table) == 0:
                rating_table = prediction
            else:
                rating_table = rating_table.set_index('tid').add(prediction.set_index('tid'), fill_value=0).reset_index()
            predictions.append(prediction)
        rating_table['rating'] /= group_size 
        rating_table['rating'] *= lambda_
        rating_table = rating_table.sort_values(by=['rating'],  ascending=False)
        
        top_n_list_svd = []
        for _, row in rating_table[:top_n_size].iterrows():
            top_n_list_svd.append(row[0])
        
        pop = pop_normalized.copy()
        
        top_n_list_pop = []
        for _, row in pop[:top_n_size].iterrows():
            top_n_list_pop.append(row[0])
            
        pop['rating'] *= (1 - lambda_)
        rating_table = rating_table.set_index('tid').add(pop.set_index('tid'), fill_value=0).reset_index()
#         rating_table = rating_table[rating_table['count'] == group_size]
        rating_table = rating_table.sort_values(by=['rating'],  ascending=False)
        rating_table = rating_table[:top_n_size]
        
        top_n_list_hybrid = []
        for _, row in rating_table.iterrows():
            top_n_list_hybrid.append(row[0])


            
        top_n_lists_hybrid.append(top_n_list_hybrid)
        top_n_lists_pop.append(top_n_list_pop)
        top_n_lists_svd.append(top_n_list_svd)
        
    top_n_lists_g_hybrid.append(top_n_lists_hybrid)
    top_n_lists_g_pop.append(top_n_lists_pop)
    top_n_lists_g_svd.append(top_n_lists_svd)

In [79]:
threshold = 0.0
cdfs_hybrid = []
for i in range(len(groups_n)):
    groups = groups_n[i]
    top_n_lists_hybrid = top_n_lists_g_hybrid[i]

    cdf_n = np.zeros(top_n_size)
    precisions = []
    for j in (range(len(groups))):

        group = groups[j]
        top_n_list_hybrid = top_n_lists_hybrid[j]
        
        cdf = []
        high_rating = 0
        
        for k in range(top_n_size):
            for l in range(len(group)):
                uid = group[l]
                tid = top_n_list_hybrid[k]
                t = test[test['uid'] == uid]
                t = t[t['tid'] == tid]
                if len(t) > 0 and t.iloc[0]['rating'] > threshold:
                    high_rating += 1
                    break
            cdf.append(high_rating)
        cdf_n = [a + b for a, b in zip(cdf_n, cdf)]
        precision = high_rating / top_n_size
        precisions.append(precision)
        
    cdf_n = [i/len(groups) for i in cdf_n]
    cdfs_hybrid.append(cdf_n)

    avg_precision = 0
    for precision in precisions:
        avg_precision += precision
    avg_precision /= len(precisions)
    print(avg_precision)

0.33951807228915665
0.45285714285714285
0.525
0.5572222222222223


In [80]:
r = np.zeros(100)
for cdf in cdfs_hybrid:
    r = [a + b for a, b in zip(r, cdf)]
r = [i/len(groups_n) for i in r]
r

[0.7815272077320271,
 1.4871151271753682,
 2.1671833855568794,
 2.7806758168203953,
 3.3801671889021287,
 3.9424584786030574,
 4.542529090721861,
 5.161210777174633,
 5.79039233858511,
 6.380286714624065,
 6.953912720478986,
 7.52846367153596,
 8.148883078099946,
 8.744240699059977,
 9.386139797284375,
 9.897036865410358,
 10.409855539373613,
 10.854908645571296,
 11.325611971696308,
 11.892895758859614,
 12.367567338952881,
 12.938688821218943,
 13.457895685305324,
 13.988702060991216,
 14.46470681259838,
 15.029458493313914,
 15.529110949291674,
 16.042917089603836,
 16.55346477485032,
 17.073017344101682,
 17.510529296673873,
 18.017759683422334,
 18.45790671844889,
 19.018690145196167,
 19.40187673772011,
 19.99405129676214,
 20.460948188357825,
 20.98861195698545,
 21.440520323050443,
 21.838107006781705,
 22.337095635288406,
 22.842323506480135,
 23.279256219015256,
 23.721733086190916,
 24.14497918413581,
 24.581391500066196,
 25.080573208585257,
 25.5804591994351,
 25.969772864

In [72]:
threshold = 0.0
cdfs_pop = []
for i in range(len(groups_n)):
    groups = groups_n[i]
    top_n_lists_svd = top_n_lists_g_svd[i]
    
    cdf_n = np.zeros(top_n_size)
    precisions = []
    for j in (range(len(groups))):

        group = groups[j]
        top_n_list_svd = top_n_lists_svd[j]

        cdf = []
        high_rating = 0

        for k in range(top_n_size):
            for l in range(len(group)):
                uid = group[l]
                tid = top_n_list_svd[k]
                t = test[test['uid'] == uid]
                t = t[t['tid'] == tid]
                if len(t) > 0 and t.iloc[0]['rating'] > threshold:
                    high_rating += 1
                    break
            cdf.append(high_rating)
        cdf_n = [a + b for a, b in zip(cdf_n, cdf)]
        precision = high_rating / top_n_size
        precisions.append(precision)
        
    cdf_n = [i/len(groups) for i in cdf_n]
    cdfs_pop.append(cdf_n)
    
    avg_precision = 0
    for precision in precisions:
        avg_precision += precision
    avg_precision /= len(precisions)
    print(avg_precision)

0.24289156626506014
0.3435714285714286
0.4026923076923076
0.45333333333333325


In [73]:
r = np.zeros(100)
for cdf in cdfs_pop:
    r = [a + b for a, b in zip(r, cdf)]
r = [i/len(groups_n) for i in r]
r

[0.6443006458066699,
 1.2146277417361755,
 1.799871647763214,
 2.3087937831913736,
 2.7734730129308445,
 3.250016549715345,
 3.725693984730129,
 4.173602836253439,
 4.675279874075055,
 5.110969519102049,
 5.57575172484811,
 6.047267825882284,
 6.43800476631802,
 6.783502140429851,
 7.188981199523369,
 7.595631610691851,
 8.042777336452035,
 8.477479515130117,
 8.863615634699972,
 9.306092501875634,
 9.651661591420629,
 10.006917781014167,
 10.404603763037498,
 10.731561778248524,
 11.04986061461965,
 11.392386395398445,
 11.7572984244671,
 12.19845131441517,
 12.49030554452241,
 12.837899215911264,
 13.186480353649028,
 13.583923606513967,
 13.983593715521426,
 14.359544404725128,
 14.757912602792121,
 15.114632522765053,
 15.490960177707166,
 15.944650396457625,
 16.313979360666107,
 16.684282919222678,
 17.000018388572606,
 17.378473601365165,
 17.711882327846183,
 18.089960574900335,
 18.440382408755905,
 18.827281654088882,
 19.18162209276667,
 19.511735587036792,
 19.8491934772055

In [74]:
threshold = 0.0
cdfs_svd = []
for i in range(len(groups_n)):
    groups = groups_n[i]
    top_n_lists_pop = top_n_lists_g_pop[i]
    
    cdf_n = np.zeros(top_n_size)
    precisions = []
    for j in (range(len(groups))):

        group = groups[j]
        top_n_list_pop = top_n_lists_pop[j]
        
        cdf = []
        high_rating = 0

        for k in range(top_n_size):
            for l in range(len(group)):
                uid = group[l]
                tid = top_n_list_pop[k]
                t = test[test['uid'] == uid]
                t = t[t['tid'] == tid]
                if len(t) > 0 and t.iloc[0]['rating'] > threshold:
                    high_rating += 1
                    break
            cdf.append(high_rating)
        cdf_n = [a + b for a, b in zip(cdf_n, cdf)]
        precision = high_rating / top_n_size
        precisions.append(precision)
        
    cdf_n = [i/len(groups) for i in cdf_n]
    cdfs_svd.append(cdf_n)
    
    avg_precision = 0
    for precision in precisions:
        avg_precision += precision
    avg_precision /= len(precisions)
    print(avg_precision)


0.32602409638554225
0.4709523809523809
0.5657692307692308
0.6644444444444444


In [62]:
r = np.zeros(100)
for cdf in cdfs_svd:
    r = [a + b for a, b in zip(r, cdf)]
r = [i/len(groups_n) for i in r]
r

[0.8437449431425335,
 1.3568872559836416,
 1.963507877664504,
 1.9724723068096561,
 2.4185183959280345,
 3.2270161231004604,
 4.075685525986731,
 4.326919399208556,
 5.031609956308752,
 5.751796563543552,
 6.402193021168925,
 6.696610618297365,
 7.322564985215587,
 7.740254056519117,
 8.035484428556718,
 8.493345175574092,
 8.741567000603146,
 8.828096267855305,
 9.641356635332539,
 10.374098959942334,
 10.439036365241185,
 10.58088765317681,
 11.217180075613811,
 11.645745987613457,
 11.912619341836209,
 12.651529193697868,
 12.732106079997646,
 13.474392073789664,
 14.006226370684201,
 14.711477779248863,
 15.353579151771921,
 16.14524765729585,
 16.385156008649986,
 17.1492673992674,
 17.415418082586758,
 18.118703752739897,
 18.26841983317887,
 18.603619606631657,
 18.80144387072098,
 19.486383261985672,
 20.0271102725922,
 20.193493755240745,
 20.87724340585786,
 21.486225120261267,
 22.05891330891331,
 22.5966687261868,
 23.29854950939288,
 23.506437839269164,
 24.13010282889801,